In [ ]:
%run -i 'nlp_implementation.py'
alltext = nlp(text_fix(open("text_files/all_fix.txt").read()))
quotes = nlp(text_fix(open("text_files/gonnadelete.txt").read()))
print("done")

In [ ]:
#prints all sentences that have quotes
for sent in alltext.sents:
    if sent._.quote==True:
        print(sent.text,"\n")

In [ ]:
#unimportant sentences
with open("gonnadelete.txt", "w+") as w:
    for sent in alltext.sents:
        if len(sent) <6:
            w.write(str(sent)+"\n")

In [ ]:
#simple subject/verb/object extraction
with open("text_files/nounchunks2.txt", "w+") as w:
    for sent in alltext.sents:
        w.write(sent.text)
        w.write("\n")
        for token in sent:
            if token.pos_=="NOUN" or token.pos_=="PROPN":
                w.write((str(token)+" ****** "))
        w.write("\n")
        for token in sent:
            if token.pos_=="VERB":
                w.write((str(token)+" ****** "))
        w.write("\n\n\n")
# find_noun(doc, "plural")
# for i in range(len(quicktext)):
#     print(quicktext[i].text, quicktext[i].pos_, quicktext[i].tag_, quicktext[i].dep_, quicktext[i].head)

In [ ]:
#intent from multiple sentences sample

doc = nlp("I have finsihed my soda. I want another one.I want another one.")
verbList = [("order", "want", "give", "make"), ("show", "find")]
dobjList = [("pizza", "pie", "pizzaz"), ("cola", "soda")]
substitutes = [("one", "it", "same", "more")] # things to replace
intent = {'verb':",'dobj':"}

for sent in doc.sents:
    for token in sent:
        if token.dep_ == 'dobj':
            verbSyns = [item for item in verbList if token.head.text in item]
            dobjSyns = [item for item in dobjList if token.text in item]
            substitute = [item for item in substitutes if token.text in item]
            if (dobjSyns != [] or substitute !=[]) and verbSyns !=[]:
                intent['verb'] = verbSyns[0][0]
            if dobjSyns !=[]:
                intent['dobj'] = dobjSyns[0][0]
            print("verbsy", verbSyns, "dobj", dobjSyns, "")
intentStr = intent['verb'] + intent['dobj'].capitalize()
print(intentStr)

In [ ]:
doc = nlp("I want to place an order for a pizza") #want intent if main words are not transitive verb or direct object
dobj, tverb = "", ""

for token in doc: #find direct object and corressponding transitive verb
    if token.dep_ == "dobj":
        dobj = token
        tverb = token.head
intentVerb = ""
verbList = ["want", "like", "need", "order"]
if tverb.text in verbList:# if tverb is what we are looking for
    intentVerb = tverb
else:
    if tverb.head.dep_ == "ROOT":#if not, then we look at root of sentence for real intent verb
        intentVerb = tverb.head
intentObj = ""
objList = ['pizza', 'cola']
if dobj.text in objList:
    intentObj = dobj
else: 
    for child in dobj.children:
        if child.dep_ == 'prep':
            intentObj = list(child.children)[0]
            break
        elif child.dep_ == 'compound':
            intentObj = child
            break
print(intentVerb.text + intentObj.text.capitalize())

In [ ]:
print("test") 

def extract_report(doc): 
    phrase = '' 
    for token in doc: 
#         print(token.text, token.pos_, spacy.explain(token.pos_), token.dep_, token.head)
        if token.pos_ =="NUM":
#             print(token.text, token.head.text)
            while True:
                phrase += " "+token.text
                token = token.head
                if token not in list(token.head.lefts):
                    phrase += " "+token.text
                    break
            
            for i in token.subtree:
                if i.dep_ == "acl":
                    phrase +=" " + i.text
                    break
            break
    phrase = phrase.strip()
    print(phrase)
#     print(doc[token.i].text)
    while True:
        token = doc[token.i].head

        if token.pos_ != "ADP":
            phrase = token.text +" "+ phrase
        if token.dep_ == "ROOT":
            break
    # print(phrase)
    for tok in token.lefts:
        if tok.dep_=="nsubj":
    #         print([tok.text for tok in tok.lefts], tok.text)
            phrase = " ".join([tok.text for tok in tok.lefts]) + ' ' + tok.text + ' ' + phrase
            break
#     print(phrase) 
    return phrase
doc = nlp("The company, whose profits reached a record high this year, largely attributed to changes in management, earned a total revenue of $4.26 million.")

phrase = extract_report(doc)
print(phrase)

# for i, token in enumerate(doc): 
#     print(token.text, token.pos_, spacy.explain(token.pos_), token.dep_, token.head)

In [ ]:
def det_destination(doc):
    for i, token in enumerate(doc):
        if token.ent_type!=0 and token.ent_type =="GPE":
            while True: #keep iterating back from gpe to either word "to" or the root
                token = token.head
                if token.text == "to":
                    return doc[i].text
                if token.head == token: #if we reach root and no to is found
                    return "failure to determine"
    return "failure to determine"



dot = nlp("I am going to the conference in Berlin")
print("It seems the user wants a ticket to ", det_destination(doc))

In [ ]:
def dep_pattern(doc): #iterate through tokens to find subject + auxiliary + Root + object pattern
    for i in range(len(doc)):
        if doc[i].dep_ == 'nsubj' and doc[i+1].dep_ == "aux" and doc[i+2].dep_ == "ROOT":
            for tok in doc[i+2].children:
                if tok.dep_== "dobj":
                    return True
    return False

def pos_pattern(doc): #iterate through tokens,check if dep_ pattern subject and object are both personal pronouns
    for token in doc:
        if token.dep_ == 'nsubj' and token.tag_ != 'PRP': #prp is personal pronoun
            return False
        if token.dep_ == 'aux' and token.tag_ != 'MD':
            return False
        if token.dep_ == 'ROOT' and token.tag_ != 'VB':
            return False
        if token.dep_ == 'dobj' and token.tag_ != 'PRP':
            return False
    return True

def pron_pattern(doc):
    plural = ["we", "us", "they", "them"]
    for token in doc:
        if token.dep_ == "dobj" and token.tag_ == "PRP":
            if token.text in plural:
                print(spacy.explain(token.pos_))
                return 'plural'
            else:
                return 'regular'
            
    return "not found"
def find_noun(sents, num):
    if num =="plural":
        taglist = ["NNS", "NNPS"]
    if num =="singular":
        taglist = ["NN", "NNP"]
    for sent in reversed(sents):
        for token in sent:
            if token.tag_ in taglist:
                current_noun = token.text
                for w in token.children:
                    if w.dep_ == "det":
                        current_noun = w.text + " " + current_noun
                return current_noun
    return "noun not found"

def gen_utterance(doc, noun):
    sent = ''
    for i, token in enumerate(doc):
        if token.dep_ == 'dobj' and token.tag_ == 'PRP':
            sent = doc[:i].text+' '+noun.lower()+' '+doc[i+1:len(doc)-2].text + 'too.'
            return sent
        
    return "failed to generate utterance"



doc = nlp("The symbols are clearly distinguishable. I can recognize them promptly.")
# displacy.serve(doc, style="dep")
# for tok in doc:
#     print(tok.text, tok.tag_,spacy.explain(tok.tag_), "****", tok.head, tok.dep_, spacy.explain(tok.dep_))
sents = list(doc.sents)
response = ''
noun = ''
for i, sent in enumerate(sents):
    if dep_pattern(sent) and pos_pattern(sent):
        noun = find_noun(sents[:i], pron_pattern(sent))
        if noun != "noun not found":
            response = gen_utterance(sents[i], noun)
            break
print(response)          



In [ ]:
token = nlp("fruits")[0]
doc = nlp("I want to buy this beautiful book at the end of the week. Sales of citrus have increased over the last year. How much do you know about this type of tree?")
similarity = {}
for i, sent in enumerate(doc.sents):
    noun_span_list = [sent[j].text for j in range(len(sent)) if sent[j].pos_ == 'NOUN']
    noun_span_str = " ".join(noun_span_list)
    noun_span_doc = nlp(noun_span_str)
    similarity.update({i:token.similarity(noun_span_doc)})
print(similarity)

In [ ]:
doc1 = nlp(u'Google Search, often referred to as simply Google, is the mostused search engine nowadays. It handles a huge number of searches each day.') 
#second sample text
doc2 = nlp(u'Microsoft Windows is a family of proprietary operating systems developed and sold by Microsoft. The company also produces a wide range of other software for desktops and servers.') 
#third sample text
doc3 = nlp(u"Titicaca is a large, deep, mountain lake in the Andes. It is known as the highest navigable lake in the world.")
docs = [doc1, doc2, doc3]
spans = {}
for j, doc in enumerate(docs):
    ner_span = [doc[i].text for i in range(len(doc)) if doc[i].ent_type != 0]
#     ner_span = [(e.text, e.label_, e.type_) for e in doc.ents]
    print(ner_span)
    ner_span = " ".join(ner_span)
    ner_span = nlp(ner_span)
    spans.update({j:ner_span})
print('doc1 is similar to doc2:',spans[0].similarity(spans[1]))
print('doc1 is similar to doc3:',spans[0].similarity(spans[2]))
print('doc2 is similar to doc3:',spans[1].similarity(spans[2]))

In [ ]:
print('england is similar to anglo:',nlp("england").similarity(nlp("india")))
print('india is similar to pakistan:',nlp("india").similarity(nlp("pakistan")))
print('england is similar to pakistan:',nlp("england").similarity(nlp("america")))

In [ ]:
doc = nlp("I can surely promise it is worth your time.")
for tok in doc:
    print(tok.text, tok.pos_,  tok.tag_,spacy.explain(tok.tag_), "****", tok.head, tok.dep_, spacy.explain(tok.dep_))


In [ ]:
sent = ''
for i, token in enumerate(doc):
    if token.tag_ == "PRP" and doc[i+1].tag_ == "MD":
        sent.append(doc[i+1].text.capitalize() + ' '+ doc[i].text +' '+ doc[i+2:].text)

doc = nlp(sent)
for i, token in (doc):
    if token.tag_ == "PRP" and 
print(sent)

In [ ]:
def find_chunk(doc):
    chunk = ""
    for i, tok in enumerate(doc):
        if tok.dep_ == "dobj":
            

In [ ]:
doc = nlp("I want to order a vetgetarian pizza")
for tok in doc:
    print(tok.text, tok.tag_,spacy.explain(tok.tag_), "****", tok.dep_, spacy.explain(tok.dep_))

In [ ]:
doc1 = nlp("we must overtake them")
doc2 = nlp("you must specify it")

for i in range(0, len(doc1)):
    if doc1[i].dep_ == doc2[i].dep_:
        print(doc1[i].text, doc2[i].text, doc1[i].dep_,)
    

In [ ]:
matcher = Matcher(nlp.vocab)
pattern = [{"DEP": "nsubj"}, {"DEP": "aux"}, {"DEP":"ROOT"}]
matcher.add("nsubjauxroot", None, pattern)
doc = nlp("We can overtake them")
matches = matcher(doc)
for match_id, start, end in matches:
    span = doc[start:end]
    print("Span: ", span.text)
    print("pos are", start ,end)

In [ ]:
def get_entities(sent):
    ent1, ent2 = "", ""
    prv_tok_dep, prv_tok_text = "",""
    prefix, modifier = "", ""
    
    for tok in nlp(sent):
        if tok.dep_ != "punct":
            print("\nprevious tok dep:", prv_tok_dep)
            print("prev tok text: ", prv_tok_text)
            print("pref:", prefix)
            print("modi:", modifier)
            
            print(tok, "...", tok.dep_)
            
            if tok.dep_ == "compound" or tok.dep_.endswith("mod") == True:
                prefix = tok.text
                if prv_tok_dep == "compound":
                    prefix = prv_tok_text + " "+ tok.text
            if tok.dep_.endswith("mod") == True:
                modifier = tok.text
                if prv_tok_dep == "compound":
                    modifier = prv_tok_text + " "+ tok.text
            if "subj" in tok.dep_:
                ent1 = modifier +" "+ prefix + " "+ tok.text
                prefix = ""
                modifier = ""
                prv_tok_dep = ""
                prv_tok_text = "" 
            if "obj" in tok.dep_:
                ent2 = modifier +" "+ prefix +" "+ tok.text
                print("fuck", modifier, "fuck", prefix, "fuck", tok.text)
#             else: 
#                 prv_tok_dep = ""
#                 prv_tok_text = "" 
            
        prv_tok_dep = tok.dep_
        prv_tok_text = tok.text
        print("\nprevious tok dep:", prv_tok_dep)
        print("prev tok text: ", prv_tok_text)
        print("pref:", prefix)
        print("modi:", modifier)
        print("\n\nentities:", ent1, "**** ",ent2)
        print("****")
                    
    
    print("\n\nentities:", ent1, "**** ",ent2)
    print("previous token stuff:", prv_tok_dep, prv_tok_text)
    print("pref:", prefix)
    print("modi:", modifier)

# get_entities(candidate_sentences[0].text)
get_entities("the real estate drawdown process is governed by real estate standard d823")



In [ ]:
text = """Mary is playing a very old violin. She is very good at it."""
facts = nlp(text)
facts._.coref_resolved